In [7]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Load tensor data
directory = './data/ffs/preprocessed/DP10_463,63636363636368-0,45818181818181825-0,13454545454545455/'
mesh_tensor = torch.load(directory + 'mesh_points.th')
sdf_tensor = torch.load(directory + 'mesh_sdf.th')
p_tensor = torch.load(directory + 'p.th')
u_tensor = torch.load(directory + 'u.th')
v_tensor = torch.load(directory + 'v.th')

# Add data to dict
dataDict = {
    'mesh': mesh_tensor,
    'sdf': sdf_tensor,
    'p': p_tensor,
    'u': u_tensor,
    'v': v_tensor,
    # 'sdf_res32': sdf_res32_tensor 
}

/tmp/ipykernel_1541916/3871987313.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mesh_tensor = torch.load(directory + 'mesh_points.th')
/tmp/ipykernel_1541916/387198731

In [1]:
from upt.datasets.ffs_dataset import ffsDataset

num_test_inputs  = 8192
num_test_outputs = 8192
# num_train_inputs  = 8192
# num_train_outputs = 16000

num_train_inputs  = num_test_inputs 
num_train_outputs = num_test_outputs

crop_values= [[-3, -2], [3, 2]]

# initialize dataset
train_dataset = ffsDataset(
    root="./data/ffs/preprocessed",
    # how many inputs to use for training
    # num_inputs=float("inf"),
    num_inputs=num_train_inputs,
    # how many outputs to use for training
    # num_outputs=float("inf"),
    num_outputs=num_train_outputs,
    # mode
    mode="train",
    # crop
    crop_values= crop_values,
)

[PosixPath('data/ffs/preprocessed/DP100_699,9999999999992-0,93818181818181645-0,46181818181818091'), PosixPath('data/ffs/preprocessed/DP10_463,63636363636368-0,45818181818181825-0,13454545454545455'), PosixPath('data/ffs/preprocessed/DP11_563,63636363636363-0,989090909090907-0,13818181818181818'), PosixPath('data/ffs/preprocessed/DP12_336,36363636363637-0,99636363636363523-0,14181818181818184'), PosixPath('data/ffs/preprocessed/DP13_372,72727272727275-0,69090909090909047-0,14545454545454548'), PosixPath('data/ffs/preprocessed/DP14_754,54545454545348-0,59636363636363643-0,14909090909090911'), PosixPath('data/ffs/preprocessed/DP15_263,63636363636368-0,27636363636363637-0,15272727272727274'), PosixPath('data/ffs/preprocessed/DP16_72,72727272727272-0,84363636363636241-0,15636363636363637'), PosixPath('data/ffs/preprocessed/DP17_200,00000000000003-0,2181818181818182-0,16000000000000003'), PosixPath('data/ffs/preprocessed/DP18_709,09090909090833-0,87272727272727124-0,16363636363636364'), Pos

In [ ]:
from torch.utils.data import DataLoader
from upt.collators.ffs_collator import ffsCollator

testvis_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=100,
    collate_fn=ffsCollator(num_supernodes=2048, deterministic=True),
)
data = [d for d in iter(testvis_dataloader)]

In [6]:
for i in range(len(data)):
    sample = data[i]
    name = sample["name"]
    print(name)

['DP100_699,9999999999992-0,93818181818181645-0,46181818181818091', 'DP10_463,63636363636368-0,45818181818181825-0,13454545454545455', 'DP11_563,63636363636363-0,989090909090907-0,13818181818181818', 'DP12_336,36363636363637-0,99636363636363523-0,14181818181818184', 'DP13_372,72727272727275-0,69090909090909047-0,14545454545454548', 'DP14_754,54545454545348-0,59636363636363643-0,14909090909090911', 'DP15_263,63636363636368-0,27636363636363637-0,15272727272727274', 'DP16_72,72727272727272-0,84363636363636241-0,15636363636363637', 'DP17_200,00000000000003-0,2181818181818182-0,16000000000000003', 'DP18_709,09090909090833-0,87272727272727124-0,16363636363636364', 'DP19_845,4545454545439-0,40727272727272734-0,1672727272727273', 'DP1_718,18181818181733-0,43636363636363645-0,10181818181818182', 'DP20_109,09090909090909-0,20363636363636364-0,17090909090909093', 'DP21_436,36363636363649-0,3636363636363637-0,17454545454545456', 'DP22_599,99999999999977-0,53818181818181832-0,17818181818181822', 'D

In [ ]:
for i in range(5):
    sample = train_dataset[i]
    # plot sparse
    x_pos, y_pos = train_dataset.denormalize_pos(sample["output_pos"]).unbind(1)
    target = train_dataset.denormalize_feat(sample["target_feat"]).clamp(0, 1)
    target = target[:,0]

    plt.figure(figsize=(10, 2))
    plt.title(sample["name"])
    plt.scatter(x_pos, y_pos, c=target, marker="s", s=1)
    plt.show()

In [1]:
from torch.utils.data import DataLoader
from upt.collators.ffs_collator import ffsCollator
from upt.datasets.ffs_dataset import ffsDataset
crop_values= [[-3, -2], [3, 2]]

test_hr_dataset = ffsDataset(
    root="./data/ffs/preprocessed",
    # how many inputs to use for training
    num_inputs=float("inf"),
    # num_inputs=8192,
    # how many outputs to use for training
    num_outputs=float("inf"),
    # num_outputs=8192,
    # mode
    mode="train",
    crop_values=crop_values,
)

In [ ]:
# get test sample
testvis_dataloader = DataLoader(
    dataset=test_hr_dataset,
    batch_size=50,
    collate_fn=ffsCollator(num_supernodes=2048, deterministic=True),
)
vis_batch = next(iter(testvis_dataloader))

RuntimeError: stack expects each tensor to be equal size, but got [46348, 2] at entry 0 and [45856, 2] at entry 1

In [5]:
# Load tensor data
directory = '../data/ffs/preprocessed/'
coords_norm = torch.load(directory + 'coords_norm.th')
vars_norm = torch.load(directory + 'vars_norm.th')

/tmp/ipykernel_373902/1446659531.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  coords_norm = torch.load(directory + 'coords_norm.th')
/tmp/ipykernel_373902/1446659531.

In [ ]:
#Save to matlab format
import scipy.io as sio

saveDir = './data/ffs/mat/'


sio.savemat(saveDir + directory.split('/')[-2] + '.mat', dataDict)